In [ ]:
metrics = train(CONFIG)
metrics

### Interpret the losses
- `d_loss` should oscillate while staying finite; collapse suggests discriminators dominating or vanishing gradients.
- `g_loss` rising steadily indicates the generator struggling to fool the discriminator; tweak learning rates or betas.
- Log the metrics JSON with experiment metadata for easier comparison across runs.
- Plot the loss curves (`pandas.DataFrame(metrics).plot()`) to verify convergence trends.

In [ ]:
samples = generate_samples(CONFIG, num_images=16, output_path=CONFIG.artifact_dir / 'notebook_samples.png')
samples.shape

### Next experiments
- Try smaller batches to see how instability manifests and adjust gradient accumulation if needed.
- Add label smoothing or noise to discriminator targets to improve training stability.
- Replace transpose convolutions with upsampling + convolutions to reduce checkerboard artefacts.
- Track Fréchet Inception Distance for objective quality measurements.